# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-22.71,84,20,3.60,CA,1702317591
1,1,lyantor,61.6195,72.1555,-32.90,95,55,3.33,RU,1702317789
2,2,grytviken,-54.2811,-36.5092,4.66,82,95,5.34,GS,1702317789
3,3,nikolayevsk-on-amure,53.1406,140.7300,-21.57,84,53,4.59,RU,1702317791
4,4,tarawa,1.4190,172.9840,26.05,94,75,0.51,KI,1702317793


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot

city_data_df['Lat'] = city_data_df['Lat'].astype(float)
city_data_df['Lng'] = city_data_df['Lng'].astype(float)
city_data_df['Humidity'] = city_data_df['Humidity'].astype(float)


# Configure the map plot with categorical legend
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    size='Humidity', 
    color='City',  # Use the color mapped to each city
    line_color='darkgray', 
    fill_alpha=0.5, 
    legend='top_right',  # Position of the legend
    tiles='OSM', 
    frame_width=1000, 
    frame_height=600,
)
# Display the map
hvplot.show(map_plot)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (
    (city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 27) &
    (city_data_df['Cloudiness']==0) &
    (city_data_df['Wind Speed'] < 4.5)
)
ideal_cities_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,kailua-kona,19.6406,-155.9956,25.89,81.0,0,2.21,US,1702317721
88,88,port elizabeth,-33.9180,25.5701,21.17,82.0,0,3.60,ZA,1702317817
134,134,hawaiian paradise park,19.5933,-154.9731,23.28,97.0,0,2.57,US,1702317827
216,216,karratha,-20.7377,116.8463,25.41,50.0,0,4.11,AU,1702317840
265,265,hilo,19.7297,-155.0900,25.34,96.0,0,2.57,US,1702317861
324,324,lompoc,34.6391,-120.4579,21.55,56.0,0,4.02,US,1702317879
457,457,port hedland,-20.3167,118.5667,23.03,64.0,0,2.57,AU,1702317898
516,516,adeje,28.1227,-16.7260,22.23,72.0,0,2.57,ES,1702317641
532,532,fada,9.5667,8.3000,22.50,26.0,0,3.47,NG,1702317917


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,kailua-kona,US,19.6406,-155.9956,81.0,
88,port elizabeth,ZA,-33.9180,25.5701,82.0,
134,hawaiian paradise park,US,19.5933,-154.9731,97.0,
216,karratha,AU,-20.7377,116.8463,50.0,
265,hilo,US,19.7297,-155.0900,96.0,
324,lompoc,US,34.6391,-120.4579,56.0,
457,port hedland,AU,-20.3167,118.5667,64.0,
516,adeje,ES,28.1227,-16.7260,72.0,
532,fada,NG,9.5667,8.3000,26.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
radius = 10000  # Reduced radius to 10 kilometers
categories="accommodation.hotel, accommodation.motel, accommodation.guest_house"
params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": 1 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Check if the response was successful
    if response.status_code == 200:
        name_address = response.json()

        
        # Attempt to extract the hotel name
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found"
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
            print(f"No hotel found for {row['City']}")
    else:
        # Print the error response (for debugging purposes)
        print(f"Failed to fetch hotels for {row['City']}. HTTP Status Code: {response.status_code}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
kailua-kona - nearest hotel: Kona Seaside Hotel
port elizabeth - nearest hotel: Waterford Hotel
No hotel found for hawaiian paradise park
hawaiian paradise park - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
hilo - nearest hotel: Dolphin Bay Hotel
lompoc - nearest hotel: Alano Club
port hedland - nearest hotel: The Esplanade Hotel
adeje - nearest hotel: Royal Garden Villas Luxury Hotel & Spa
No hotel found for fada
fada - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
12,kailua-kona,US,19.6406,-155.9956,81.0,Kona Seaside Hotel
88,port elizabeth,ZA,-33.9180,25.5701,82.0,Waterford Hotel
134,hawaiian paradise park,US,19.5933,-154.9731,97.0,No hotel found
216,karratha,AU,-20.7377,116.8463,50.0,Karratha International Hotel
265,hilo,US,19.7297,-155.0900,96.0,Dolphin Bay Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

hotel_df['Lat'] = hotel_df['Lat'].astype(float)
hotel_df['Lng'] = hotel_df['Lng'].astype(float)

# Configure the map plot using hvplot

map_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity', color='City', 
                                  hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'], 
                                  fill_alpha=0.5, tiles='OSM', frame_width=700, frame_height=500)

# Display the map
hvplot.show(map_plot)